In [1]:
import multiprocessing as mp
from multiprocessing import Process
import zmq

In [2]:
context = zmq.Context()

In [3]:
class Worker(Process):
    def __init__(self, worker_id):
        super().__init__()
        self.worker_id = worker_id
        print(self.worker_id,'init')
    def run(self):
        # recieve work
        print(self.worker_id,'run')
        consumer_receiver = context.socket(zmq.PULL)
        consumer_receiver.connect("tcp://127.0.0.1:5558")
        # send work
        consumer_sender = context.socket(zmq.PUSH)
        consumer_sender.connect("tcp://127.0.0.1:5559")
        print(f"worker[{self.worker_id}] stand by ...")
        
        for _ in range(5):
            work = consumer_receiver.recv_json()
            data = work['num']
            print(f"worker[{self.worker_id}] data: {data}")
            result = { 'consumer' : self.worker_id, 'num' : data}
            consumer_sender.send_json(result)

In [4]:
# 开启两个消费者（worker）进程
context = zmq.Context()
worker1 = Worker(0)
worker2 = Worker(1)
p1 = Process(name = "worker1",target = worker1.run())
p2 = Process(name = "worker2",target = worker2.run())
p1.start()
p2.start()

0 init
1 init
0 run
worker[0] stand by ...
worker[0] data: 0
worker[0] data: 1
worker[0] data: 2
worker[0] data: 3
worker[0] data: 4
1 run
worker[1] stand by ...
worker[1] data: 0
worker[1] data: 1
worker[1] data: 2
worker[1] data: 3
worker[1] data: 4
